# Procedure

This script is an HR-3DXRD adaptation of the workflow presented in https://pyfai.readthedocs.io/en/master/usage/tutorial/Goniometer/Rotation-Pilatus100k/Multi120_Pilatus100k.html (access date May 31st, 2021). 


!!! The current version of the script requires the following input files present in the working directory:
- Calibrant file, '.D'
- Image files, '.tif, .edf, etc.'
- Control points file for each image, '.npt'
- Detector calibration file for each image, '.PONI'


Step 1 - Initial calibration 1: 
    Define experimentally measured distance. Calibration guesses of individual detector positions should be provided (as control points, '.npt.' and '.PONI' files). Initial calibration of sample-to-detector distance and PONI parameters. All tilts are set to zero.
    
Step 2 - Initial calibration 2:  
    Inherit the refined calibration from the previous step. Calibrate sample-to-detector distance and PONI parameters with new expression. All tilts are set to zero.  
    
Step 3 - Complete calibration: 
    Inherit the refined calibration from previous step. Calibrate all global parameters. Initial guess of both tilts are set to zero.
    
Step 4 & 5 -  Refinement:
    Inherit all parameters from previous step. Repeat the 3rd step with non-zero initial guesses for both detector tilts.

Step 6 - Analysis:
    1D azimuthal integration and 2D polar decomposition of the HR-3DXRD compound image. Refined motor positions are calculated. Conversion from pyFAI to FABLE/ImageD11 geometry and producing the parameter files, '.par'.

# Initialization

Definitions of working directory, input images, motor positions and its reader function, and definitions of calibrant, detector, etc.

In [ ]:
%pylab nbagg

import time, pyFAI
print("Using pyFAI version", pyFAI.version)

start_time = time.time()

#Loading required libraries
import os
import random
import fabio
from pyFAI.goniometer import GeometryTransformation, GoniometerRefinement, Goniometer
from pyFAI.gui import jupyter

# Image/working directory:

im_dir = '/home/esrf/kutsal/desy_oct21_analysis_ann2_ep1_nf2/single_panel_ponis/testing/'
os.chdir(im_dir)

# Loading images...

all_files = []
image_files = []

for f in os.listdir(im_dir):
    if f.split('.')[-1] == 'tif':
        if f.split('.')[0].split('_')[-3] == str(2) :
            continue
        if f.split('.')[0].split('_')[-3] == str(5) :
            continue  
        if f.split('.')[0].split('_')[-3] == str(12) :
            continue
        else:
            all_files.append(im_dir + f);

image_files = [i for i in all_files if i.split('.')[-1] == 'tif']
image_files.sort()
            
print('Number of loaded images: '+str(len(image_files)))

In [ ]:
# Testing: Did we loaded the images successfully?

#fimg = fabio.open(image_files[1])

#plt.figure()
#plt.imshow(fimg.data,cmap="gray", vmin=50, vmax=350)
#plt.title('Raw Image of '+fimg.filename )
#plt.colorbar()

In [ ]:
# Definition of the motor position function

# The refinement procedure requires a function that reads
# the experimental motor position of the acquired image. 
# Works with '.edf' files, or any format with headers 
# containing its motor positions. In case, please change the 
# reading format according to your header's data structure.


#def get_pos(header):
#    #print("get_pos: ", img_basename)
#    #for_header = fabio.open(img_basename);
#    #header = for_header.header;
#    motor_mne = header["motor_mne"].split(" ");
#    motor_pos = header["motor_pos"].split(" ");
#    a = dict(zip(motor_mne, motor_pos));
#    d2ty = float(a.get("d2ty"));
#    d2tz = float(a.get("d2tz"));    
#    return d2ty , d2tz

In [ ]:
# Definition of the position function - manual way: No 
# header info in .tif images. Experimental motor positions 
# are provided manually and they are called from a list, 
# instead of image header.

motorpos_list = [(0, (5.8, 0)),(1, (0.535, -5.775)),(2, (-5.7, 1.066)),(3, (-1.587, -5.578)),
                 (4, (5.408, 2.095)),(5, (-4.931, -3.05)),(6, (4.286, -3.907)),
                 (7, (2.585, 5.19)),(8, (-3.495, 4.628)),(9, (0.535, 5.77)),
                 (10, (5.7, -1.065)),(11, (2.585, -5.192)),(12, (-4.931, 3.05)),
                 (13, (5.408, -2.095)),(14, (-1.587, 5.578)),(15, (-3.495, -4.628)),
                 (16, (4.286, 3.907))]

def get_pos(img_basename):
    i=img_basename.split('/')[-1].split('_')[1]
    k,yz = motorpos_list[int(i)]
    if k == int(i):        
        return yz[0]*1e-3, yz[1]*1e-3  #returns in meters

In [ ]:
# Definition of the employed detector and the calibrant

frelon = pyFAI.detector_factory("frelon")

# PCO edge is not avaiable. Going for a way-around... 
# Please comment out the part below, if your detector is
# available in pyFAI's database.

frelon.pixel1=1.3e-6
frelon.pixel2=1.3e-6
frelon.shape=(2540,2140)
frelon.MAX_SHAPE=(2540,2140)  
print(frelon)


# Manual definition of the calibrant.
# The user-made calibrant files may require further manual definition 
# of the d-spacings and the wavelength... Please comment out the part 
# below, if your calibrant/material is available in pyFAI's database.

d_list = [2.027, 1.433, 1.170]
wavelength = 3.2627e-11

calib = pyFAI.calibrant.Calibrant()
calib.load_file("aFe.d")
calib.dSpacing= d_list
calib.wavelength= wavelength

print("Calibrant loaded: ", calib.filename)
print("d-spacings: " + str(calib.dSpacing))
print("Wavelength to be used: " + str(calib.wavelength))

In [ ]:
# If the input images may lose their header information during the
# through-stack summation of pre-processing. This step re-intorduces
# the header information to the summed images, from the respective
# experimental diffraction images. This step is required  if only the 
# images have header info...

#def copy_headers(fn):
#    eta1 = fabio.open(fn)
#    orig = eta1.header["merged_file_0000"]
#    header = fabio.open("/data/id06/inhouse/2019/MKutsal/ID11_HEA_x10_data/
#                   HEA_x10_eta_all/"+os.path.split(eta1.header["merged_file_0000"])[-1]).header
#    eta1.header.update(header)
#    print(eta1.header)
#    eta1.write(fn)




#copy_headers("maxi_eta1.edf")
#copy_headers("maxi_eta2.edf")
#copy_headers("maxi_eta3.edf")
#copy_headers("maxi_eta4.edf")

# Step 1: Initial calibration I

Distance= constant, tilts y&z=0. PONI=PONI(posY, posZ). -> Fit the distance and PONI only.

In [ ]:
# Definition of the goniometer translation function:
# The detector moves in y_L and z_L axes, (presumably) with no rotation

goniotrans = GeometryTransformation(param_names = ["dist", "poni1_scale", "posY_offset", 
                                                   "poni2_scale", "posZ_offset", "rot1",
                                                   "rot2"],
                                    pos_names = ["posY", "posZ"],
                                    dist_expr= "dist",
                                    poni1_expr="-posZ * poni1_scale + posZ_offset", 
                                    poni2_expr="-posY * poni2_scale + posY_offset", 
                                    rot1_expr="0.0", 
                                    rot2_expr="0.0", 
                                    rot3_expr="0.0")

# Initial guesses of the refinement
# The parameter order is the same as the param_names of the output file.

param = {"dist":0.02885, # in meters
         "poni1_scale": 1.0, 
         "posZ_offset": 0.0, 
         "poni2_scale": 1.0, 
         "posY_offset": 0.0, 
         "rot1":0.0, # in radians
         "rot2":0.0,
         "rot3":0.0         
          }

# Boundary conditions of the refinement

bounds = {"dist": (0.025, 0.035),
          "poni1_scale": (-1.0,1.0),
          "poni2_scale": (-1.0,1.0),
          "posZ_offset": (-1.0,1.0),
          "posY_offset": (-1.0,1.0)
         }

# Initialization of the refinement object

gonioref = GoniometerRefinement(param, #initial guess
                                pos_function=get_pos,
                                trans_function=goniotrans,
                                bounds=bounds,
                                detector=frelon, wavelength=wavelength)


print("Empty refinement object:")
print(gonioref)

In [ ]:
# Populating the goniometer refinement object with control point and PONI files:

for image in image_files:
    fimg = fabio.open(image)
    
    basename = os.path.splitext(fimg.filename)[0]
    
    which_eta = 'pos_'+basename.split('_')[-3]
    print(basename, which_eta)
    
    sg =gonioref.new_geometry(which_eta, image=fimg.data, metadata=fimg.filename, 
                              control_points=which_eta+"_max.npt", geometry=which_eta+"_max.poni", calibrant=calib)
    sg.control_points.calibrant = calib
    sg.geometry_refinement.wavelength = wavelength

    print(sg.label," Detector Position: ",sg.get_position())
    print(sg.geometry_refinement)
    print()


print("Filled refinement object:")
print(gonioref)


In [ ]:
# Display all images with associated calibration:

nimg = len(gonioref.single_geometries)
fig,ax = subplots(nimg, 1, figsize=(8,nimg*3))
for i, sg in enumerate(gonioref.single_geometries.values()):
    jupyter.display(sg=sg, ax=ax[i])

In [ ]:
# Repeat the refinement for determining the local minimum

i=0
while i<6:
    gonioref.refine2(maxiter=100000000)
    i=i+1

In [ ]:
# Check the calibration results...

def print_geo(key):
    print("expected")
    print(gonioref.single_geometries[key].geometry_refinement)
    print("refined")
    print(gonioref.get_ai(gonioref.single_geometries[key].get_position()))
    
print_geo("pos_0")
print_geo("pos_10")
print_geo("pos_13")
print_geo("pos_16")

# Step 2: Initial calibration II

Distance=Distance(posY, posZ) and PONI=PONI(posY, posZ). Tilts y&z=0. -> Fit the distance and PONI only.




In [ ]:
goniotrans2 = GeometryTransformation(param_names = ["dist", "poni1_scale", "posY_offset", 
                                                   "poni2_scale", "posZ_offset",
                                                   "dist1_scale", "dist2_scale"],
                                    pos_names = ["posY", "posZ"],
                                    dist_expr= "dist + dist1_scale * -posZ + dist2_scale * - posY",
                                    poni1_expr="-posZ * poni1_scale + posZ_offset", 
                                    poni2_expr="-posY * poni2_scale + posY_offset", 
                                    rot1_expr="0.0", 
                                    rot2_expr="0.0", 
                                    rot3_expr="0.0")




#Defines the bounds for some variables
bounds2 = {"dist": (0.025, 0.035),
          "poni1_scale": (-1.0,1.0),
          "poni2_scale": (-1.0,1.0),
          "posZ_offset": (-1.0,1.0),
          "posY_offset": (-1.0,1.0),
          "dist1_scale":(-5.0,5.0),
          "dist2_scale":(-5.0,5.0),
         }




param2 = (gonioref.nt_param(*gonioref.param))._asdict()

# Initial guesses for dist_scale parameters
param2["dist1_scale"]= 0.0
param2["dist2_scale"]= 0.0



gonioref2 = GoniometerRefinement(param2, #initial guesses from previous steps results
                                pos_function=get_pos,
                                trans_function=goniotrans2,
                                bounds=bounds,
                                detector=frelon, wavelength=wavelength)




gonioref2.single_geometries = gonioref.single_geometries.copy()
print(gonioref2)

In [ ]:
for image in image_files:
    fimg = fabio.open(image)
    
    basename = os.path.splitext(fimg.filename)[0]
    
    which_eta = 'pos_'+basename.split('_')[-3]
    print(basename, which_eta)
    
    
    
    
    sg =gonioref2.new_geometry(which_eta, image=fimg.data, metadata=fimg.filename, control_points=which_eta+"_max.npt", 
                              geometry=gonioref.get_ai(gonioref.single_geometries[which_eta].get_position()), 
                               calibrant=calib)
    sg.control_points.calibrant = calib
    sg.geometry_refinement.wavelength = wavelength

    
    print(sg.label," Detector Position: ",sg.get_position())
    print(sg.geometry_refinement)
    print()

    
    

print("Filled refinement object:")
print(gonioref2)

In [ ]:
# Repeat the refinement for determining the local minimum


i=0
while i<6:
    gonioref2.refine2("slsqp", eps=1e-13, maxiter=100000000, ftol=1e-12)
    i=i+1

In [ ]:
# Check the calibration results...

def print_geo2(key):
    print("expected")
    print(gonioref.single_geometries[key].geometry_refinement)
    print("refined")
    print(gonioref2.get_ai(gonioref.single_geometries[key].get_position()))

    

    
    
print_geo2("pos_0")
print_geo2("pos_10")
print_geo2("pos_13")
print_geo2("pos_16")

# Step 3: Complete calibration

Distance=Distance(posY, posZ) and PONI=PONI(posY, posZ). Tilt-y and tilt-z=constant. -> Fit all global parameters.

In [ ]:
goniotrans3 = GeometryTransformation(param_names = ["dist", "poni1_scale", "posY_offset", 
                                                   "poni2_scale", "posZ_offset", "rot1", "rot2", 
                                                   "dist1_scale", "dist2_scale"],
                                    pos_names = ["posY", "posZ"],
                                    dist_expr= "dist + dist1_scale * -posZ + dist2_scale * - posY",
                                    poni1_expr="-posZ * poni1_scale + posZ_offset", 
                                    poni2_expr="-posY * poni2_scale + posY_offset", 
                                    rot1_expr="rot1", 
                                    rot2_expr="rot2", 
                                    rot3_expr="0.0")



#Defines the bounds for some variables
bounds3 = {"dist": (0.025, 0.035),
          "poni1_scale": (-1.0,1.0),
          "poni2_scale": (-1.0,1.0),
          "posZ_offset": (-1.0,1.0),
          "posY_offset": (-1.0,1.0),
          "rot1": (-0.1,0.1),
          "rot2": (-0.1,0.1),
          "dist1_scale":(-5.0,5.0),
          "dist2_scale":(-5.0,5.0),
         }

param3 = (gonioref2.nt_param(*gonioref2.param))._asdict()

# Initial parameters for tilt-z and tilt-y, respectively
param3["rot1"]= 0.0174533 #in radians
param3["rot2"]= 0.0174533


            
gonioref3 = GoniometerRefinement(param3, #initial guesses from previous steps results
                                pos_function=get_pos,
                                trans_function=goniotrans3,
                                bounds=bounds3,
                                detector=frelon, wavelength=wavelength)

gonioref3.single_geometries = gonioref2.single_geometries.copy()
print(gonioref3)

In [ ]:
for image in image_files:
    fimg = fabio.open(image)
    
    basename = os.path.splitext(fimg.filename)[0]
    
    
    
    which_eta = 'pos_'+basename.split('_')[-3]
    print(basename, which_eta)
    
    
    sg =gonioref3.new_geometry(which_eta, image=fimg.data, metadata=fimg.filename, control_points=which_eta+"_max.npt", 
                              geometry=gonioref2.get_ai(gonioref2.single_geometries[which_eta].get_position()), 
                               calibrant=calib)
    
  
    sg.control_points.calibrant = calib
    sg.geometry_refinement.wavelength = wavelength

    
    
    print(sg.label," Detector Position: ",sg.get_position())
    print(sg.geometry_refinement)
    print()


print("Filled refinement object:")
print(gonioref3)

In [ ]:
# Repeat the refinement for determining the local minimum

i=0
while i<6:
    gonioref3.refine2("slsqp", eps=1e-13, maxiter=100000000, ftol=1e-12)
    i=i+1

In [ ]:
# Check the calibration results...

def print_geo3(key):
    print("expected")
    print(gonioref.single_geometries[key].geometry_refinement)
    print("refined")
    print(gonioref3.get_ai(gonioref.single_geometries[key].get_position()))
    
      
    
    
print_geo3("pos_0")
print_geo3("pos_10")
print_geo3("pos_13")
print_geo3("pos_16")

# Step 4: Refinement I

Repeat of Step 3.

In [ ]:
goniotrans4 = GeometryTransformation(param_names = ["dist", "poni1_scale", "posY_offset", 
                                                   "poni2_scale", "posZ_offset", "rot1", "rot2", 
                                                   "dist1_scale", "dist2_scale"],
                                    pos_names = ["posY", "posZ"],
                                    dist_expr= "dist + dist1_scale * -posZ + dist2_scale * - posY",
                                    poni1_expr="-posZ * poni1_scale + posZ_offset", 
                                    poni2_expr="-posY * poni2_scale + posY_offset", 
                                    rot1_expr="rot1", 
                                    rot2_expr="rot2", 
                                    rot3_expr="0.0")




#Defines the bounds for some variables
bounds4 = {"dist": (0.025, 0.035),
          "poni1_scale": (-1.0,1.0),
          "poni2_scale": (-1.0,1.0),
          "posZ_offset": (-1.0,1.0),
          "posY_offset": (-1.0,1.0),
          "rot1": (-0.1,0.1),
          "rot2": (-0.1,0.1),
          "dist1_scale":(-5.0,5.0),
          "dist2_scale":(-5.0,5.0),
         }


param4 = (gonioref3.nt_param(*gonioref3.param))._asdict()


gonioref4 = GoniometerRefinement(param4, #initial guesses from previous steps results
                                pos_function=get_pos,
                                trans_function=goniotrans4,
                                bounds=bounds4,
                                detector=frelon, wavelength=wavelength)


gonioref4.single_geometries = gonioref3.single_geometries.copy()
print(gonioref4)

In [ ]:
for image in image_files:
    fimg = fabio.open(image)
    
    
    
    basename = os.path.splitext(fimg.filename)[0]
    
    
    which_eta = 'pos_'+basename.split('_')[-3]
    print(basename, which_eta)
    
    
    
    sg =gonioref4.new_geometry(which_eta, image=fimg.data, metadata=fimg.filename, control_points=which_eta+"_max.npt", 
                              geometry=gonioref3.get_ai(gonioref3.single_geometries[which_eta].get_position()),
                               calibrant=calib)
    
    
    
    sg.control_points.calibrant = calib
    sg.geometry_refinement.wavelength = wavelength
 
    print(sg.label," Detector Position: ",sg.get_position())
    print(sg.geometry_refinement)
    print()

    
    

print("Filled refinement object:")
print(gonioref4)

In [ ]:
# Repeat the refinement for determining the local minimum


i=0
while i<6:
    gonioref4.refine2("slsqp", eps=1e-13, maxiter=100000000, ftol=1e-12)
    i=i+1

In [ ]:
# Check the calibration results...

def print_geo4(key):
    print("expected")
    print(gonioref.single_geometries[key].geometry_refinement)
    print("refined")
    print(gonioref4.get_ai(gonioref.single_geometries[key].get_position()))
 
    
print_geo4("pos_0")
print_geo4("pos_10")
print_geo4("pos_13")
print_geo4("pos_16")

# Step 5: Refinement II

Repeat of Step 3.

In [ ]:
param5 = (gonioref4.nt_param(*gonioref4.param))._asdict()

# Bounds are inherited from previous calibration step

gonioref5 = GoniometerRefinement(param5, #initial guesses from previous steps results
                                pos_function=get_pos,
                                trans_function=goniotrans4,
                                bounds=bounds4,
                                detector=frelon, wavelength=wavelength)

gonioref5.single_geometries = gonioref4.single_geometries.copy()
print(gonioref5)

In [ ]:
for image in image_files:
    fimg = fabio.open(image)
    
    basename = os.path.splitext(fimg.filename)[0]
    
    which_eta = 'pos_'+basename.split('_')[-3]
    print(basename, which_eta)
    
    
    sg =gonioref5.new_geometry(which_eta, image=fimg.data, metadata=fimg.filename, control_points=which_eta+"_max.npt", 
                              geometry=gonioref4.get_ai(gonioref4.single_geometries[which_eta].get_position()), 
                              calibrant=calib)
    sg.control_points.calibrant = calib
    sg.geometry_refinement.wavelength = wavelength

    print(sg.label," Detector Position: ",sg.get_position())
    print(sg.geometry_refinement)
    print()


print("Filled refinement object:")
print(gonioref5)

In [ ]:
# Repeat the refinement for determining the local minimum

i=0
while i<6:
    gonioref5.refine2("slsqp", eps=1e-13, maxiter=100000000, ftol=1e-12)
    i=i+1

In [ ]:
# Check the calibration results...

def print_geo5(key):
    print("expected")
    print(gonioref.single_geometries[key].geometry_refinement)
    print("refined")
    print(gonioref5.get_ai(gonioref.single_geometries[key].get_position()))
    
print_geo4("pos_0")
print_geo4("pos_10")
print_geo4("pos_13")
print_geo4("pos_16")

# Save the refined calibration in ASCII format
gonioref5.save("multipanel_geometry_all.json")

print("Refined calibration execution time: "+str(time.time() - start_time)+" seconds")

# Step 6: Analysis

- 1D azimuthal integration & 2D polar decomposition plots. 
- Calculation of refined motor positions and derivation of errors with respect to experimental observations. 
- Conversion from pyFAI to FABLE/ImageD11 geometry, production of ImageD11-type parameter files.

In [ ]:
#Reload all images, for extrapolating the refined calibration to the omitted images...

all_files=[]
image_files=[]


for f in os.listdir(im_dir):
    if f.split('.')[-1] == 'tif':
            all_files.append(im_dir + f);

print('Number of loaded images: '+str(len(all_files)))


image_files = [i for i in all_files if i.split('.')[-1] == 'tif']
image_files.sort()


# Restore the translation table setting (i.e. the refined calibration) from the file

transtable = Goniometer.sload("multipanel_geometry_all.json")
transtable.detector.pixel1= 1.3e-6
transtable.detector.pixel2= 1.3e-6
transtable.detector.MAX_SHAPE=(2540,2140)
print("Translation table: \n",transtable)

# Complete motor positions list, including the images omitted from the calibration

mt = [(0.0058, 0.0), (0.0057, -0.001065), (0.002585, -0.005192), (-0.004931, 0.00305), (0.005408, -0.002095), 
      (-0.001587, 0.005578), (-0.003495, -0.004628), (0.004286, 0.003907), (0.000535, -0.005775), (-0.0057, 0.001066), 
      (-0.001587, -0.005578), (0.005408, 0.002095), (-0.004931, -0.00305), (0.004286, -0.003907), (0.002585, 0.00519), 
      (-0.003495, 0.004628), (0.000535, 0.00577)]

print("Motor pos: \n", mt)


# Create a multi-geometry object for all images in this refinement set:

multigeo = transtable.get_mg(mt)
multigeo.radial_range=(7, 16)
print(multigeo)

from pyFAI.method_registry import IntegrationMethod

#Selection of the methods for integrating
method = IntegrationMethod.parse("full", dim=1)
method2d = IntegrationMethod.select_one_available(("pseudo","histogram","cython"), dim=2)


# Integrate the set of images in a single run:

res = multigeo.integrate1d([fabio.open(fn).data for fn in image_files], 10000, method=method)

filename = "multipanel_geometry_all_1dint.xy"
f1 = open(filename, "w")
f1.write("\n")
f1.write("#  2theta intensity \n")
fmt = "%6f "*2 + " \n"
for i in range(len(res[0])):
    f1.write(fmt % ( res[0][i], res[1][i]))
f1.close()

# Display the result using matplotlib
fig, ax = subplots()
ax.plot(*res)
ax.set_xlabel(res.unit.label)
ax.set_ylabel("Intensity")
#ax.set_xlim(17, 22)
#ax.set_title("Zoom on the two first rings")

fig, ax = subplots(2, 1, figsize=(12,16))
jupyter.plot1d(*res,ax=ax[0])

res2d = multigeo.integrate2d([fabio.open(fn).data for fn in image_files], 1000,360)
jupyter.plot2d(res2d, ax=ax[1])

In [ ]:
#Display all images with associated calibration:

nimg = len(gonioref5.single_geometries)
fig,ax = subplots(nimg, 1, figsize=(8,nimg*3))
for i, sg in enumerate(gonioref5.single_geometries.values()):
    jupyter.display(sg=sg, ax=ax[i])

# Generation of FABLE/ImageD11 parameter files

Conversion from pyFAI to ImageD11 geometry. Takes motor positons and the refined calibration result, and generates '.par' files, containing global parameters for each panel in ImageD11-formatting.

In [ ]:
# Statics: Load refined calibration parameters
panel_fov = frelon.shape #transposed!!
dist = gonioref5.param[0]
poni1_scale = gonioref5.param[1]
posY_offset = gonioref5.param[2]
poni2_scale = gonioref5.param[3]
posZ_offset = gonioref5.param[4]
rot1 = gonioref5.param[5]
rot2 = gonioref5.param[6]
rot3 = 0.0
dist1_scale = gonioref5.param[7]
dist2_scale = gonioref5.param[8]

In [ ]:
# Save .par files in ImageD11 formatting

os.mkdir('ImD11_pars')
os.chdir('/home/esrf/kutsal/desy_oct21_analysis_ann2_ep1_nf2/single_panel_ponis/testing/ImD11_pars/')

mt_ref=[]

del_x = []
del_z = []
del_y = []


print('The refined motor positions calculated from the refined calibration result\n')
print('Panel no.  RMP_y (mm)  RMP_z (mm)\n')

for i,(posY, posZ) in motorpos_list:
    
    posY = posY*1e-3 #now in m
    posZ = posZ*1e-3 #now in m
    
    dist_exp = eval(goniotrans4.dist_expr)
    poni1_exp = eval(goniotrans4.poni1_expr)
    poni2_exp = eval(goniotrans4.poni2_expr)
     
    ImageD11 = {}
    ImageD11["wavelength"] = float(calib.wavelength*1e10)
    ImageD11["distance"] = dist_exp/cos(rot1)/cos(rot2)*1e6 #Convert to um for ImD11 formatting
    ImageD11["y_size"] = frelon.pixel2*1e6 #Convert to um for ImD11 formatting
    ImageD11["z_size"] = frelon.pixel1*1e6 #Convert to um for ImD11 formatting
    ImageD11["y_center"] = (poni2_exp - dist_exp * tan(rot1))/float(frelon.pixel2)
    ImageD11["z_center"] = (poni1_exp + dist_exp * tan(rot2)/cos(rot1))/float(frelon.pixel1)
    ImageD11["tilt_x"] = rot3
    ImageD11["tilt_y"] = rot2
    ImageD11["tilt_z"] = -rot1
    ImageD11["o11"] = 1
    ImageD11["o12"] = 0
    ImageD11["o21"] = 0
    ImageD11["o22"] = -1
    
    #Below are for my own convenience. Take out for general distribution...
    ImageD11["cell__a"] = 2.866
    ImageD11["cell__b"] = 2.866
    ImageD11["cell__c"] = 2.866
    ImageD11["cell_alpha"] = 90.0
    ImageD11["cell_beta"] = 90.0
    ImageD11["cell_gamma"] = 90.0
    ImageD11["cell_lattice_[P,A,B,C,I,F,R]"] = 'I'
    ImageD11["omegasign"] = 1
    ImageD11["chi"] = 0.0
    ImageD11["fit_tolerance"] = 0.005
    
    outfile = 'pos_'+str(i)+'_pycalib.par'
    f1 = open(outfile,'w')
    
        
    for key in sorted(ImageD11):
        #print("{0:s}: {1:g}".format(key, ImageD11[key]))
        fmt = str(key)+' '+ str(ImageD11[key])+'\n'
        f1.write(fmt)
    f1.close()
    
    
    mtref_z = (ImageD11["z_center"]-panel_fov[1]*0.5)* frelon.pixel1*-1
    mtref_y = (ImageD11["y_center"]-panel_fov[0]*0.5)* frelon.pixel1*-1
    
    mt_ref.append((i,(mtref_y,mtref_z)))
    
    del_x.append((dist_exp-dist)*1e6) #in microns
    del_z.append((mtref_z-posZ)*1e6) #in microns
    del_y.append((mtref_y-posY)*1e6) #in microns
    
    print i, mtref_y*1e3, mtref_z*1e3 #in millimeters

    
# Calculated error of detector motor pos. in 3D
print('Average error in x_L position: ',np.mean(del_x)*1e3,' mm with st.dev. of ',np.std(del_x)*1e3)
print('Average error in y_L position: ',np.mean(del_y)*1e3,' mm with st.dev. of ',np.std(del_y)*1e3)
print('Average error in x_L position: ',np.mean(del_z)*1e3,' mm with st.dev. of ',np.std(del_z)*1e3)
    

ylab_list=[]
zlab_list=[]
for i,(y,z) in mt_ref:
    ylab_list.append(y)
    zlab_list.append(z)
    

# Calculated FoV of the compound image
fov_big_h = (abs(max(ylab_list))+abs(min(ylab_list)))/frelon.pixel1+panel_fov[1]
fov_big_v = (abs(max(zlab_list))+abs(min(zlab_list)))/frelon.pixel1+panel_fov[0]

print('FoV of the compound image: hor= ',fov_big_h,',  vertical= ',fov_big_v' pixels\n')
print
print("Total execution time: "+str(time.time() - start_time)+" seconds")